In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import calendar


# Allow Markdown characters in the print statement
# this can make the output more print ready
def printmd(string):
    display(Markdown(string))

In [2]:
conn = sqlite3.connect("twitterTesla.db")
c = conn.cursor()

# Create the Tweets Table
# quoted_status and reweeted_status are bool
# but sqlite doesn't have bool actual data types so I am using 0 for false and 1 for true
sql_create = """
CREATE TABLE IF NOT EXISTS tweets ( 
    tweet_id INTEGER PRIMARY KEY,
    tweet TEXT,
    created_at INTEGER,
    retweet_count INTEGER,
    favorite_count INTEGER,
    quoted_status INTEGER,
    retweeted_status INTEGER,
    source TEXT
);
"""

# remove any data from a previous run
sql_delete = "DELETE FROM tweets"

try:
    c.execute(sql_create)
    c.execute(sql_delete)
    conn.commit()
finally:
    conn.rollback()
    c.close()
    conn.close()

In [4]:
conn = sqlite3.connect("twitterTesla.db")
c = conn.cursor()

# Create tesla table to hold stock prices

sql_create = """
CREATE TABLE IF NOT EXISTS tesla (
    id INTEGER PRIMARY KEY,
    date INTEGER,
    open DECIMAL(6,2),
    close DECIMAL(6,2),
    high DECIMAL(6,2),
    low DECIMAL(6,2)
);
"""

# remove any data from a previous run
sql_delete = "DELETE FROM tesla"

try:
    c.execute(sql_create)
    c.execute(sql_delete)
    conn.commit()
finally:
    conn.rollback()
    c.close()
    conn.close()